In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from collections import Counter

import warnings
warnings.filterwarnings('ignore')

In [ ]:
emit_data_data = pd.read_csv('~/Desktop/Founders_Pledge/affectable_emissions_active_repo/Affectable-Emissions/data_files/all_vars_w_disagg_considered_vals.csv', index_col = 0).drop(columns = ['region'])
affectability = pd.read_csv('~/Desktop/Founders_Pledge/affectable_emissions_active_repo/Affectable-Emissions/data_files/affectability_data/affectability_v2.csv').drop(columns = {'Unnamed: 0', 'Var.1', 'region', 'expectable.2100', 'commit.2018', 'commit.2019', 'commit.2020'})

In [ ]:
# put columns in same order so that .mult works properly
affectability1 = affectability.reindex(sorted(affectability.columns), axis=1)
emit_data_data1 = emit_data_data.reindex(sorted(emit_data_data.columns), axis=1)

In [ ]:
# make sure the columns of the dfs are the same before multiplying by index
assert affectability1.columns.equals(emit_data_data1.columns) == True

# multiply the dfs columnwise, weighting emissions by their affectibility factor
affectable_emissions = affectability1.iloc[:,3:].mul(emit_data_data1.iloc[:,3:], axis = 1)
affectable_emissions[['Country', 'RCP', 'SSP']] = affectability1.iloc[:,:3]

# put country, rcp, ssp columns at the front of the df
country = affectable_emissions.pop('Country')
ssp = affectable_emissions.pop('SSP')
rcp = affectable_emissions.pop('RCP')
affectable_emissions.insert(0, country.name, country)
affectable_emissions.insert(1, ssp.name, ssp)
affectable_emissions.insert(2, rcp.name, rcp)

#affectable_emissions.to_csv('~/Desktop/Founders_Pledge/affectable_emissions_active_repo/Affectable-Emissions/data_files/affectability_data/emit_data_weighted_by_affectability.csv')

In [ ]:
# elif planned;
# elif committed:
# section could be done using a map instead of a loop
# data = defaultdict{}
# categories = [planned, commit,...]
# for var in flipped.index:
#      for c in categories:
#           if c in vars:
#              data[c].append(var[-4:], )

# then can delete the loops
# use dict agg to skip loop, or could maybe have index over the columns?

In [ ]:
flipped = affectable_emissions.transpose()
data_per_scenario = {}
all_years = np.arange(2018, 2100, dtype = int)

for column in flipped.columns:
    current_scenario = flipped[column][['Country','SSP','RCP']]
    
    scenario_vals = []
    committed_vals = []
    expectable_vals = []
    afec_consid_commit = []
    
    gas_vals = []
    coal_vals = []
    steel_vals = []

    for var in flipped.index:
        if 'planned' in var:
            scenario_vals.append([var[-4:], flipped[column][var]])                
        
        elif 'commit' in var:   
            committed_vals.append([var[-4:], flipped[column][var]])
       
        elif 'expectable' in var:
            expectable_vals.append([var[-4:], flipped[column][var]])
        
        elif 'gas' in var:
            if int(var[-4:]) > 2020 and int(var[-4:]) != 2075:
                gas_vals.append([var[-4:], flipped[column][var]])
                
        elif 'coal' in var:
            if int(var[-4:]) > 2020:
                coal_vals.append([var[-4:], flipped[column][var]])
            
        elif 'steel' in var:
            if int(var[-4:]) > 2020:
                steel_vals.append([var[-4:], flipped[column][var]])
                
    data_per_scenario[tuple(current_scenario)] = {'affectable_planned': np.array(scenario_vals, dtype = float), 
                                           'affectable_committed': np.array(committed_vals, dtype = float), 
                                           'coal': np.array(coal_vals, dtype = float),
                                           'gas': np.array(gas_vals, dtype = float),
                                           'steel': np.array(steel_vals, dtype = float),
                                            #where expectable is [planned - (considered + committed)]
                                           'affectable_expectable': np.array(expectable_vals, dtype = float)}
for scenario in data_per_scenario:
    data = data_per_scenario[scenario]
        
    keys_commit = list((Counter(dict(data_per_scenario[scenario]['gas'])) + Counter(dict(data_per_scenario[scenario]['coal'])) + Counter(dict(data_per_scenario[scenario]['steel']))).keys())
    values_commit = list((Counter(dict(data_per_scenario[scenario]['gas'])) + Counter(dict(data_per_scenario[scenario]['coal'])) + Counter(dict(data_per_scenario[scenario]['steel']))).values())
    
    #considered emissions multiplies values by 2 in order to estimate the "true" value of considered emit
    #same implementation of 'considered_emissions' as in stacked plots
    data['affectable_considered'] = (np.array([(k,v) for k,v in dict(zip(keys_commit, values_commit * 2)).items()]))
    
for scenario in data_per_scenario:
    data = data_per_scenario[scenario]
    
    keys_sum = list((Counter(dict(data_per_scenario[scenario]['affectable_considered'])) + Counter(dict(data_per_scenario[scenario]['affectable_committed']))).keys())
    values_sum = list((Counter(dict(data_per_scenario[scenario]['affectable_considered'])) + Counter(dict(data_per_scenario[scenario]['affectable_committed']))).values())
    
    data['sum_consid_commit'] = np.array([(k,v) for k,v in dict(zip(keys_sum, values_sum)).items()])

### Raw Affectability

In [ ]:
# ### custom_lines = [Line2D([0], [0], color='darksalmon', lw=2), 
#                 Line2D([0], [0], color='sandybrown', lw=2), Line2D([0], [0], color='cadetblue', lw=2)]
# for i, scenario in enumerate(data_per_scenario):
#     plt.figure(figsize=(10,3.2))

#     for key in data_per_scenario[scenario]:
        
#         #considered emissions multiplied by 2 in order to estimate the "true" value of considered emit
#         #same implementation of 'considered_emissions' as in stacked plots
#         if 'affectable_considered' in key:
#             plt.plot(data_per_scenario[scenario][key][:, 0], data_per_scenario[scenario][key][:, 1].astype(float) * 2, label = str(key), color = 'darksalmon')
#         if 'affectable_committed' in key:
#             plt.plot(data_per_scenario[scenario][key][:, 0], data_per_scenario[scenario][key][:, 1].astype(float), label = str(key), color = 'sandybrown')
#         if 'affectable_planned' in key:
#             plt.plot(data_per_scenario[scenario][key][:, 0], data_per_scenario[scenario][key][:, 1].astype(float), label = str(key), color = 'cadetblue')
#             plt.legend(custom_lines, ['Affectable Considered Emissions','Affectable Committed Emissions', 'Affectable Expected Emissions'], loc = 'upper right', bbox_to_anchor=(1, 1))
              
#     plt.hlines(0, 2021, 2100, color = 'black', linestyle = '--', label = 'Negative Emissions')    
#     plt.xlim(2021,2100)
    
#     plt.xlabel('Year')
#     plt.ylabel('Emissions (MtCO2/yr)')
#     plt.title('Affectable Emissions emissions till 2100 (affectable fraction of expected, committed, and considered emissions) ' + str(scenario))
#     plt.savefig('/Users/Violet/Desktop/Founders_Pledge/affectable_emissions_active_repo/Affectable-Emissions/data_files/affectability_plots/' + 'affectable' + str(scenario) + '.jpeg', transparent = True, bbox_inches = 'tight', dpi = 1000)
    
#     #('China', 'SSP2', '6.0') and ('China', 'SSP3', '4.5') not included in the affectability data, 
#     # plus the last row is all NaNs... so:
#     #i==25 runs all scenarios
#     if i==25: 
#         break

### Stacked/Filled Affectability Plots

In [ ]:
# linearly interpolate between x ranges

In [ ]:
# custom_lines = [Line2D([0], [0], color='darksalmon', lw=2), 
#                 Line2D([0], [0], color='fuchsia', lw=2), 
#                 Line2D([0], [0], color='cadetblue', lw=2)]

# for i, scenario in enumerate(data_per_scenario):
#     plt.figure(figsize=(10,3.2))

#     for key in data_per_scenario[scenario]:
        
#         if 'affectable_planned' in key:
#             plt.plot(data_per_scenario[scenario][key][:, 0], data_per_scenario[scenario][key][:, 1].astype(float), label = str(key), color = 'cadetblue')
#             plt.fill_between(data_per_scenario[scenario][key][:, 0], data_per_scenario[scenario][key][:, 1], color = 'cadetblue', capstyle = 'round')
        
#         if 'sum_consid_commit' in key:
#             plt.fill_between(data_per_scenario[scenario][key][:, 0], data_per_scenario[scenario][key][:, 1], color = 'lightcoral', capstyle = 'round', alpha = 0.9)
        
#         if 'affectable_committed' in key:
#             plt.fill_between(data_per_scenario[scenario][key][:, 0], data_per_scenario[scenario][key][:, 1], color = 'blue')
#             plt.legend(custom_lines, ['Affectable Considered Emissions','Affectable Committed Emissions', 'Affectable Expected Emissions'], loc = 'upper right', bbox_to_anchor=(1, 1))
                   
#     plt.hlines(0, 2021, 2100, color = 'black', linestyle = '--', label = 'Negative Emissions')    
#     plt.xlim(2021,2100)
    
#     plt.xlabel('Year')
#     plt.ylabel('Emissions (MtCO2/yr)')
#     plt.title('Affectable Emissions emissions till 2100 (affectable fraction of expected, committed, and considered emissions) ' + str(scenario))
#     plt.savefig('/Users/Violet/Desktop/Founders_Pledge/affectable_emissions_active_repo/Affectable-Emissions/data_files/affectability_plots/' + 'affectable_stacked' + str(scenario) + '.jpeg', transparent = True, bbox_inches = 'tight', dpi = 1000)
    
#     #('China', 'SSP2', '6.0') and ('China', 'SSP3', '4.5') not included in the affectability data, 
#     # plus the last row is all NaNs... so:
#     #i==25 runs all scenarios
#     if i==25: 
#         break

In [ ]:
custom_lines = [Line2D([0], [0], color='darksalmon', lw=2), 
                Line2D([0], [0], color='fuchsia', lw=2), 
                Line2D([0], [0], color='cadetblue', lw=2)]

for i, scenario in enumerate(data_per_scenario):
    plt.figure(figsize=(10,3.2))

    for key in data_per_scenario[scenario]:
        
        if 'affectable_planned' in key:
            plt.plot(data_per_scenario[scenario][key][:, 0], data_per_scenario[scenario][key][:, 1].astype(float), label = str(key), color = 'cadetblue')
            plt.fill_between(data_per_scenario[scenario][key][:, 0], data_per_scenario[scenario][key][:, 1], color = 'cadetblue', capstyle = 'round')
        
        if 'sum_consid_commit' in key:
            plt.fill_between(data_per_scenario[scenario][key][:, 0], data_per_scenario[scenario][key][:, 1], color = 'lightcoral', capstyle = 'round', alpha = 0.9)
        
        if 'affectable_committed' in key:
            plt.fill_between(data_per_scenario[scenario][key][:, 0], data_per_scenario[scenario][key][:, 1], color = 'blue')
            plt.legend(custom_lines, ['Affectable Considered Emissions','Affectable Committed Emissions', 'Affectable Expected Emissions'], loc = 'upper right', bbox_to_anchor=(1, 1))
                   
    plt.hlines(0, 2021, 2100, color = 'black', linestyle = '--', label = 'Negative Emissions')    
    plt.xlim(2021,2100)
    
    plt.xlabel('Year')
    plt.ylabel('Emissions (MtCO2/yr)')
    plt.title('Affectable Emissions emissions till 2100 (affectable fraction of expected, committed, and considered emissions) ' + str(scenario))
    plt.savefig('/Users/Violet/Desktop/Founders_Pledge/affectable_emissions_active_repo/Affectable-Emissions/data_files/affectability_plots/' + 'affectable_stacked' + str(scenario) + '.jpeg', transparent = True, bbox_inches = 'tight', dpi = 1000)
    
    #('China', 'SSP2', '6.0') and ('China', 'SSP3', '4.5') not included in the affectability data, 
    # plus the last row is all NaNs... so:
    #i==25 runs all scenarios
    if i==25: 
        break